In [1]:
import numpy as np
import pandas as pd
from re import sub
from time import time
import pickle

from comet_ml import Experiment

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras_tqdm import TQDMNotebookCallback as ktqdm
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.initializers import RandomUniform, RandomNormal
from keras.layers.advanced_activations import LeakyReLU

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# experiment = Experiment(api_key="CndJ3YmXyZcxmsV8EccJEuu9C",
#                         project_name="NN_Thesis", workspace="paologalligit")

In [3]:
df = pd.read_csv('fifa19.csv')

In [4]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [5]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)
    

Individuare eventuali outlier nella colonna value

In [6]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score= (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [7]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [8]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [9]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

Calcolo correlazione tra i valori per scegliere colonne significative

In [10]:
df_corr = df.corr()

# fig = plt.figure(figsize=(50,20))
# ax = fig.add_subplot(111)
# cax = ax.matshow(df_corr,cmap='coolwarm', vmin=-1, vmax=1)
# fig.colorbar(cax)

# ticks = np.arange(0,len(df_corr.columns),1)
# ax.set_xticks(ticks)
# ax.set_xticklabels(df_corr.columns)
# plt.xticks(rotation=90)
# ax.set_yticks(ticks)
# ax.set_yticklabels(df_corr.columns)

# plt.show()

In [11]:
labels = []
for label in df_corr:
#     if df_corr['Value'][label] < 0 or df_corr['Value'][label] > 0.5: labels.append(label)
    if df_corr['Value'][label] > 0.55: labels.append(label)
        
df_flt = df[labels]        
df_flt.head()      

,Overall,Potential,Value,Wage,LCM,CM,RCM,Reactions,Release Clause
41,88,88,4000000.0,77000.0,NaN,NaN,NaN,79.0,7400000.0
102,85,85,9000000.0,38000.0,70.0,70.0,70.0,85.0,15300000.0
108,85,85,9000000.0,57000.0,63.0,63.0,63.0,83.0,17100000.0
152,84,84,4200000.0,95000.0,63.0,63.0,63.0,80.0,6900000.0
201,83,83,13000000.0,70000.0,NaN,NaN,NaN,78.0,24700000.0


Mescolo le righe del dataset

In [12]:
df_flt = df_flt.sample(frac=1)

train_slice = int(len(df_flt) * 0.8)

train = df_flt[:train_slice]
test = df_flt[train_slice:]

In [13]:
y_train = train.loc[:, ['Value']]
X_train = train.drop(columns='Value')

y_test = test.loc[:, ['Value']]
X_test = test.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [14]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train)
X_full = imputer.transform(X_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_train)
y_full = imputer.transform(y_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_test)
X_test_full = imputer.transform(X_test)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_test)
y_test_full = imputer.transform(y_test)

Scalo i valori, sia per i caratteri che per il target

In [15]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

scaler = StandardScaler().fit(X_test_full)
X_test_scaled = scaler.transform(X_test_full)

scaler_test = StandardScaler().fit(y_test_full)
y_test_scaled = scaler_test.transform(y_test_full) 

### Salvo i dati di training e testing

In [17]:
tools = {
    'X_scaled': X_scaled,
    'y_scaled': y_scaled,
    'X_test_scaled': X_test_scaled,
    'y_test_scaled': y_test_scaled,
    'y_test_full': y_test_full,
    'scaler_train': scaler_train
}

with open('dataset_tools', 'wb') as file:
    pickle.dump(tools, file)